In [1]:
import torch
from diffusers import AnimateDiffPipeline, LCMScheduler, MotionAdapter
from diffusers.utils import export_to_gif

from diffusers.utils.import_utils import is_xformers_available

device = "cuda:1"
dtype = torch.float16
adapter = MotionAdapter.from_pretrained("wangfuyun/AnimateLCM", torch_dtype=dtype).to(device)
pipe = AnimateDiffPipeline.from_pretrained("emilianJR/epiCRealism", motion_adapter=adapter, torch_dtype=dtype).to(device)

if is_xformers_available():
    print("enable xformers memory efficient attention")
    pipe.unet.enable_xformers_memory_efficient_attention()
else:
    print("install xformers to enable memory efficient attention")

pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config, beta_schedule="linear")

pipe.load_lora_weights("wangfuyun/AnimateLCM", weight_name="AnimateLCM_sd15_t2v_lora.safetensors", adapter_name="lcm-lora")
# pipe.set_adapters(["lcm-lora"], [0.8])

# pipe.enable_vae_slicing()
# pipe.enable_model_cpu_offload()


/home/yiming/anaconda3/envs/DiT/lib/python3.11/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
The config attributes {'conv_in_channels': None} were passed to MotionAdapter, but are not expected and will be ignored. Please verify your config.json configuration file.


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/home/yiming/anaconda3/envs/DiT/lib/python3.11/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
The config attributes {'center_input_sample': False, 'flip_sin_to_cos': True, 'freq_shift': 0, 'mid_block_type': 'UNetMidBlock2DCrossAttn', 'only_cross_attention': False, 'attention_head_dim': 8, 'dual_cross_attention': False, 'class_embed_type': None, 'addition_embed_type': None, 'num_class_embeds': None, 'upcast_attention': False, 'resnet_time_scale_shift': 'default', 'resnet_skip_time_act': False, 'resnet_out_scale_factor': 1.0, 'time_embedding_type': 'positional', 'time_embedding_dim': None, 'time_embedding_act_fn': None, 'timestep_post_act': None, 'time_cond_proj_dim': None, 'conv_in_kernel': 3, 'conv_out_kernel': 3, 'projection_class_embeddings_input_dim': None, 'class_embeddings_concat': False, 'mid_b

enable xformers memory efficient attention


In [2]:
prompts = ["Sunset over the sea.",
           "A quiet beach at dawn, the waves gently lapping at the shore and the sky painted in pastel hues.",
           "Time-lapse of a coastal landscape transitioning from sunrise to nightfall.",
           "A serene underwater scene featuring a sea turtle swimming.",
           "Yellow and black tropical fish dart through the sea.",
           "a dynamic interaction between the ocean and a large rock.",
           "The dynamic movement of tall, wispy grasses swaying in the wind.",
           "Slow pan upward of blazing oak fire in an indoor fireplace.",
           "A serene waterfall cascading down moss-covered rocks."]

for prompt in prompts:
    output = pipe(prompt=prompt,
                negative_prompt="bad quality, worse quality, low resolution",
                    num_frames=16,
                    guidance_scale=2.0,
                    num_inference_steps=6,
                    generator=torch.Generator("cpu").manual_seed(0),
                )
        # export_to_gif(output.frames[0], "AnimateLCM_{}.gif".format(prompt[:30].replace(" ", "_")))
    
# output = pipe(
#     prompt="A space rocket with trails of smoke behind it launching into space from the desert, 4k, high resolution",
#     negative_prompt="bad quality, worse quality, low resolution",
#     num_frames=16,
#     guidance_scale=2.0,
#     num_inference_steps=6,
#     generator=torch.Generator("cpu").manual_seed(0),
# )
# frames = output.frames[0]
# export_to_gif(frames, "animatelcm.gif")

  0%|          | 0/6 [00:00<?, ?it/s]

RuntimeError: CUDA error: invalid configuration argument
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
